In [ ]:
pip install accelerate -U

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 41.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.1
    Uninstalling huggingface-hub-0.19.1:
      Successfully uninstalled huggingface-hub-0.19.1


In [ ]:
# import the necessary libraries
import numpy as np
import pandas as pd
from google.colab import drive
import torch
import torch.nn as nn
import transformers
from transformers import AlbertForMultipleChoice, AlbertTokenizer, AdamW
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Load data from .npy file
data = np.load("/content/gdrive/MyDrive/SP-train.npy", allow_pickle=True)[()]
len(data)

507

In [ ]:
import numpy as np
import torch

seed = 42  # You can choose any seed value
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
split_index = 407

training_data = data[:split_index]
validation_data = data[split_index:]

# Extract questions, options, and correct_indices for training and validation sets
questions = [entry['question'] for entry in training_data]
options = [entry['choice_list'] for entry in training_data]
correct_indices = [entry['label'] for entry in training_data]

validation_questions = [entry['question'] for entry in validation_data]
validation_options = [entry['choice_list'] for entry in validation_data]
validation_correct_indices = [entry['label'] for entry in validation_data]

In [ ]:
df=pd.DataFrame(validation_data)
df

,0
0,"{'id': 'SP-164_CR', 'question': 'Mark was in t..."
1,"{'id': 'SP-165', 'question': 'Cleopatra and An..."
2,"{'id': 'SP-165_SR', 'question': 'Cleopatra and..."
3,"{'id': 'SP-165_CR', 'question': 'Two chickens ..."
4,"{'id': 'SP-167', 'question': 'There is a bomb ..."
...,...
95,"{'id': 'SP-207_SR', 'question': 'Ten pears hun..."
96,"{'id': 'SP-207_CR', 'question': 'There were tw..."
97,"{'id': 'SP-208', 'question': 'The more you tak..."
98,"{'id': 'SP-208_SR', 'question': 'The more you ..."


No charts were generated by quickchart


In [ ]:
print(questions[1])
print(options[1])
print(correct_indices[1])

The six daughters of Mr. and Mrs. Mustard each have one brother. However, the family only consists of nine people; how is that possible?
['Some brothers were not loved by family and moved away.', 'Some daughters get married and have their own family.', 'Each daughter shares the same brother.', 'None of above.']
2


In [ ]:
len(training_data)

407

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

In [ ]:
from transformers import AutoTokenizer
import numpy as np

# Tokenize questions and options for training set
tokenized_training_data = []
for i in range(len(questions)):
    choices = options[i]
    input_ids = []
    attention_mask = []

    for choice in choices:
        # Combine the question and choice
        text = questions[i] + " " + choice

        # Tokenize and encode the text
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

        input_ids.append(inputs['input_ids'])
        attention_mask.append(inputs['attention_mask'])

    tokenized_training_data.append({
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'correct_index': correct_indices[i]
    })

# Tokenize questions and options for validation set
tokenized_validation_data = []
for i in range(len(validation_questions)):
    choices = validation_options[i]
    input_ids = []
    attention_mask = []

    for choice in choices:
        # Combine the question and choice
        text = validation_questions[i] + " " + choice

        # Tokenize and encode the text
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

        input_ids.append(inputs['input_ids'])
        attention_mask.append(inputs['attention_mask'])

    tokenized_validation_data.append({
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'correct_index': validation_correct_indices[i]
    })

# Save tokenized data as numpy arrays
np.save('tokenized_training_data.npy', tokenized_training_data)
np.save('tokenized_validation_data.npy', tokenized_validation_data)

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch
import numpy as np

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        # Extract questions, options, and correct indices
        input_ids = np.array([feature["input_ids"] for feature in features])
        attention_mask = np.array([feature["attention_mask"] for feature in features])
        labels = np.array([feature["correct_index"] for feature in features])

        batch_size = len(input_ids)
        num_choices = input_ids.shape[1]

        # Flatten features
        flattened_features = {
            "input_ids": input_ids.reshape(batch_size * num_choices, -1),
            "attention_mask": attention_mask.reshape(batch_size * num_choices, -1),
        }

        # Pad and reshape
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Reshape the batch
        batch = {
            k: v.view(batch_size, num_choices, -1) for k, v in batch.items()
        }
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
from transformers import AutoTokenizer

# Tokenize questions and options for training set
tokenized_training_data = []
max_sequence_length = 128  # Define your desired sequence length

for i in range(len(questions)):
    choices = options[i]
    input_ids = []
    attention_mask = []

    for choice in choices:
        # Combine the question and choice
        text = questions[i] + " " + choice

        # Tokenize and encode the text with padding and truncation
        inputs = tokenizer(
            text,
            return_tensors='pt',
            padding='max_length',  # Pad to the specified sequence length
            truncation=True,
            max_length=max_sequence_length,
        )

        input_ids.append(inputs['input_ids'])
        attention_mask.append(inputs['attention_mask'])

    tokenized_training_data.append({
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'correct_index': correct_indices[i]
    })

# Tokenize questions and options for validation set
tokenized_validation_data = []
for i in range(len(validation_questions)):
    choices = validation_options[i]
    input_ids = []
    attention_mask = []

    for choice in choices:
        # Combine the question and choice
        text = validation_questions[i] + " " + choice

        # Tokenize and encode the text with padding and truncation
        inputs = tokenizer(
            text,
            return_tensors='pt',
            padding='max_length',  # Pad to the specified sequence length
            truncation=True,
            max_length=max_sequence_length,
        )

        input_ids.append(inputs['input_ids'])
        attention_mask.append(inputs['attention_mask'])

    tokenized_validation_data.append({
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'correct_index': validation_correct_indices[i]
    })

# Save tokenized data as numpy arrays
np.save('tokenized_training_data.npy', tokenized_training_data)
np.save('tokenized_validation_data.npy', tokenized_validation_data)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = np.mean(predictions == labels)
    original = (np.sum(predictions[0::3] == labels[0::3]) * 3) / len(labels)
    semantic = (np.sum(predictions[1::3] == labels[1::3]) * 3) / len(labels)
    context = (np.sum(predictions[2::3] == labels[2::3]) * 3) / len(labels)
    average = np.sum(predictions == labels) / len(labels)
    orisem = 0
    orisemcon = 0
    for idx in range(0, len(labels), 3):
        orisem += (predictions[idx] == labels[idx] and predictions[idx + 1] == labels[idx + 1])
        orisemcon += (predictions[idx] == labels[idx] and predictions[idx + 1] == labels[idx + 1] and predictions[idx + 2] == labels[idx + 2])
    orisem = (orisem * 3) / len(labels)
    orisemcon = (orisemcon * 3) / len(labels)
    return {'average': average, 'original': original, 'semantic': semantic, 'context': context, 'orisem': orisem, 'orisemcon': orisemcon}


In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.data.data_collator import DataCollatorWithPadding

# Load your tokenized training and validation data
tokenized_training_data = np.load('tokenized_training_data.npy', allow_pickle=True)
tokenized_validation_data = np.load('tokenized_validation_data.npy', allow_pickle=True)

# Define a custom dataset class for multiple-choice questions
class MultipleChoiceDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'input_ids': self.data[idx]['input_ids'],
            'attention_mask': self.data[idx]['attention_mask'],
            'labels': self.data[idx]['correct_index']
        }

# Create DataLoader for training and validation
batch_size = 16  # Adjust as needed
train_dataset = MultipleChoiceDataset(tokenized_training_data)
train_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

validation_dataset = MultipleChoiceDataset(tokenized_validation_data)
validation_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForMultipleChoice.from_pretrained("albert-base-v2")

# Define training arguments
training_args = TrainingArguments(
    output_dir="albert",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,  # Set to True if you want to push the model to the Hugging Face Model Hub
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=train_data_collator,  # Use the data collator for training
    train_dataset=train_dataset,  # Use the dataset for training
    eval_dataset=validation_dataset,  # Use the dataset for validation
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Average,Original,Semantic,Context,Orisem,Orisemcon
1,No log,0.883318,0.580000,0.600000,0.570000,0.570000,0.300000,0.270000
2,No log,0.852223,0.690000,0.660000,0.660000,0.750000,0.360000,0.360000
3,No log,0.694503,0.720000,0.690000,0.720000,0.750000,0.450000,0.420000


TrainOutput(global_step=78, training_loss=0.4243639432466947, metrics={'train_runtime': 116.1905, 'train_samples_per_second': 10.509, 'train_steps_per_second': 0.671, 'total_flos': 29176697797632.0, 'train_loss': 0.4243639432466947, 'epoch': 3.0})

In [ ]:
# Evaluate the model on the validation dataset
eval_results = trainer.evaluate()

# Print the final evaluation metrics
print("Final Evaluation Metrics:")
for key, value in eval_results.items():
    print(f"{key}: {value}")

Final Evaluation Metrics:
eval_loss: 0.6945032477378845
eval_average: 0.72
eval_original: 0.69
eval_semantic: 0.72
eval_context: 0.75
eval_orisem: 0.45
eval_orisemcon: 0.42
eval_runtime: 3.6149
eval_samples_per_second: 27.663
eval_steps_per_second: 1.936
epoch: 3.0
